# Invoking and Testing the Vector Store Inference Service (Optional)

Welcome to the third part of our tutorial series on building a question-answering application over a corpus of private documents using Large Language Models (LLMs). In our previous Notebooks, you've embarked on the journey of transforming unstructured text data into structured vector embeddings and deploying an Inference Service to serve the Vector Store that holds these embeddings.

In this optional Notebook, you will focus on invoking the Vector Store Inference Service you've created and testing its performance. This is an essential step, as it allows us to verify the functionality of our service and observe how it performs in practice. Throughout this Notebook, we will guide you on how to construct suitable requests, communicate with the service, and interpret the responses.

By the end of this Notebook, you will gain practical insights into the workings of the Vector Store Inference Service and will be well-prepared to integrate it into a larger system, alongside the Large Language Model Inference Service that we will create in the subsequent Notebook.

Let's get started! As always, let's import the libraries you'll need:

In [ ]:
import json
import requests
import ipywidgets as widgets

To invoke the Vector Store inference service you'll need to authenticate yourself. To this end, fill the following boxes with your credentials and the EzAF cluster domain that the inference services lives to retrieve a token you can use for subsequent requests:

In [ ]:
# Add heading
heading = widgets.HTML("<h2>Serving Credentials</h2>")
display(heading)

ezaf_env_input = widgets.Text(description='EZAF Env:')
namespace_input = widgets.Text(description='Namespace:')
username_input = widgets.Text(description='Username:')
password_input = widgets.Password(description='Password:')
submit_button = widgets.Button(description='Submit')
success_message = widgets.Output()

ezaf_env = None
namespace = None
username = None
password = None

def submit_button_clicked(b):
    global ezaf_env, namespace, username, password
    ezaf_env = ezaf_env_input.value
    namespace = namespace_input.value
    username = username_input.value
    password = password_input.value
    with success_message:
        success_message.clear_output()
        print("Credentials submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)

# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

# Display inputs and button
display(ezaf_env_input, namespace_input, username_input, password_input, submit_button, success_message)

In [ ]:
EZAF_ENV = ezaf_env 
token_url = f"https://keycloak.{EZAF_ENV}.com/realms/UA/protocol/openid-connect/token"

data = {
    "username" : username,
    "password" : password,
    "grant_type" : "password",
    "client_id" : "ua-grant",
}

token_responce = requests.post(token_url, data=data, allow_redirects=True, verify=False)

token = token_responce.json()["access_token"]

Finally, you need to construct the URL you'll hit and define the payload for the POST request you'll send. For this example. you'll be using the V1 inference protocol, which is described below:

| API          | Verb | Path                          | Request Payload   | Response Payload                  |
|--------------|------|-------------------------------|-------------------|-----------------------------------|
| List Models  | GET  | /v1/models                    |                   | {"models": [<model_name>]}        |
| Model Ready  | GET  | /v1/models/<model_name>       |                   | {"name": <model_name>,"ready": $bool} |
| Predict      | POST | /v1/models/<model_name>:predict | {"instances": []}** | {"predictions": []}              |
| Explain      | POST | /v1/models/<model_name>:explain | {"instances": []}** | {"predictions": [], "explanations": []} |

** Payload is optional

We want to invoke the `predict` API. So let's use a simple query to test our service:

In [ ]:
NAMESPACE = namespace
DEPLOYMENT_NAME = "vectorstore"
MODEL_NAME = DEPLOYMENT_NAME
SVC = f'{DEPLOYMENT_NAME}-predictor-default.{NAMESPACE}.{EZAF_ENV}.com'
URL = f"https://{SVC}/v1/models/{MODEL_NAME}:predict"

print(URL)

In [ ]:
data = {
  "instances": [{
      "question": "Who's Ada Lovelace?"
  }]
}

headers = {"Authorization": f"Bearer {token}"}

response = requests.post(URL, json=data, headers=headers, verify=False)

In [ ]:
response.text

# Conclusion and Next Steps

Well done! Through this Notebook, you've successfully interacted with and tested the Vector Store Inference Service. You've learned how to construct and send requests to the service and how to interpret the responses. This hands-on experience is crucial as it provides a practical understanding of the service's operation, preparing you for real-world applications.

In the next Notebook, you will extend our question-answering system by creating an Inference Service for the Large Language Model (LLM). The LLM Inference Service will work in conjunction with the Vector Store Inference Service to provide comprehensive and accurate answers to user queries.